In [7]:
from tenacity import retry, stop_after_attempt 
import requests
from bs4 import BeautifulSoup
import pandas as pd 

base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

@retry(stop=stop_after_attempt(3))
def get_html(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    return soup

all_data = []
max_page = 40

for page in range(1, max_page + 1):
    
    url = base_url.format(page)
    
    soup = get_html(url)
    
    items = soup.find_all("div",{"class":"cassetteitem"})
    print("page", page, "items", len(items))
    
    for item in items:
        stations = item.findAll("div",{"class":"cassetteitem_detail-text"})
        
        for station in stations:
            base_data = {}
            
            base_data["名称"] = item.find("div", {"class":"cassetteitem_content-title"}).getText().strip()
            base_data["カテゴリ"] = item.find("div", {"class":"cassetteitem_content-label"}).getText().strip()
            base_data["住所"] = item.find("li", {"class":"cassetteitem_detail-col1"}).getText().strip()
            base_data["最寄り駅"] = station.getText().strip()
            base_data["築年数"] = item.find("li", {"class":"cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()
            base_data["構造"] = item.find("li", {"class":"cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()
        
            tbodys = item.find("table", {"class":"cassetteitem_other"}).findAll("tbody")
            
            for tbody in tbodys:
                data = base_data.copy()
                
                data["階数"] = tbody.findAll("td")[2].getText().strip()
    
                data["賃料"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
                data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()
                
                data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()
                data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()
                
                data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
                data["専有面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()
            
                all_data.append(data)  

# convert to dataframe
df = pd.DataFrame(all_data)
df.to_csv("suumo01.csv", encoding="utf-8")
